In [1]:
"""

This code gathers all the data together and stores it in a mongodb:
In order to run this notebook. First a list of Citys must be selected
And from there the BIT website must be scraped city by city to generate 
a set of bands. 

"""

import gather_data as gd
import pandas as pd
import re
from pymongo import MongoClient
import requests

        

mc = MongoClient()  # Connect to the MongoDB server using default settings
capstone_db = mc['stranger_scraping']  # Use (or create) a database called 'election_predictions'
location_info = capstone_db['events']

#this was my attempt to gather the venue data right after the call
#it has missing values though an will be easier to deal with 
#after the fact

'''def clean_event_to_store(e):
    #print(e)
    e['venue_gps'] = e['venue']['latitude'] +','+ e['venue']['longitude']
    e['country'] = e['venue']['country']
    e['region'] = e['venue']['region']
    e['city'] = e['venue']['city']
    e['venue_name'] = e['venue']['name']
    e['venue_long'] = e['venue']['longitude']
    e['venue_lat'] = e['venue']['latitude']
    del e['venue']
                                                            
    return e'''

def api_tour_call(band):
    
    # these next lines of code remove chars the the BIT api does not like
    band_name = band
    band_name = band_name.replace(' ','%20')
    band_name = band_name.replace( '/', '%252F')
    band_name = band_name.replace( '?', '%253F')
    band_name = band_name.replace( '*', '%252A')
    band_name = band_name.replace( '"', '%2&C')
    
    #print('api_call', band_name)
    #ther are a small number of bands with names to long for the api
    #these cases are just droped from the model 
    
    try:
        result = requests.get("https://rest.bandsintown.com/artists/" +
                            band_name +
                          "/events?app_id=2eafc16df6cb4f3db3bb365ca0d91f6e",)
        return (result.json())
    except: 
        return band

    
    
def store_event_data(band_names):
    bad_bands = []
    c = MongoClient()
    capstone_db = mc['bit_scraping']
    event_info  = capstone_db['events']
    for i, band in enumerate(band_names):
        if i%20 == 1:
            print('storing tour', i , 'of', len(band_names))
        tour = api_tour_call(band) 
        if tour == band:
            bad_bands.append(band)
        else:
            for j, event in enumerate(tour):
                event_info.update_one({'id':event['id']},
                                     {'$set': event}, upsert=True)
    return bad_bands

In [2]:
bands = pd.read_csv('junk_bands.csv')['band_names']
#b = get_band_tour(bands[0]).json()

In [3]:
store_event_data(bands)

storing tour 1 of 489
storing tour 21 of 489
storing tour 41 of 489
storing tour 61 of 489
storing tour 81 of 489
storing tour 101 of 489
storing tour 121 of 489
storing tour 141 of 489
storing tour 161 of 489
storing tour 181 of 489
storing tour 201 of 489
storing tour 221 of 489
storing tour 241 of 489
storing tour 261 of 489
storing tour 281 of 489
storing tour 301 of 489
storing tour 321 of 489
storing tour 341 of 489
storing tour 361 of 489
storing tour 381 of 489
storing tour 401 of 489
storing tour 421 of 489
storing tour 441 of 489
storing tour 461 of 489
storing tour 481 of 489


['Musician Jesse Cook',
 'Blitzen Trapper',
 'Life in the Fast Lane "Eagles Tribute"',
 'Bob Seger The Silver Bullet Band',
 'Daniel Champagne Music']

In [39]:
tour[0]

{'offers': [{'type': 'VIP',
   'url': 'https://www.bandsintown.com/t/1007083232?app_id=2eafc16df6cb4f3db3bb365ca0d91f6e&came_from=267&custom=true',
   'status': 'available'},
  {'type': 'Tickets',
   'url': 'https://www.bandsintown.com/t/1007083232?app_id=2eafc16df6cb4f3db3bb365ca0d91f6e&came_from=267',
   'status': 'available'}],
 'venue': {'name': 'Greek Theatre (w/ Steve Miller Band)',
  'country': 'United States',
  'region': 'CA',
  'city': 'Los Angeles',
  'latitude': '34.1194881',
  'longitude': '-118.2962909'},
 'datetime': '2018-08-11T19:00:00',
 'on_sale_datetime': '2018-02-17T18:00:00',
 'description': '',
 'lineup': ['Peter Frampton', 'Steve Miller Band'],
 'id': '1007083232',
 'artist_id': '1955',
 'url': 'https://www.bandsintown.com/e/1007083232?app_id=2eafc16df6cb4f3db3bb365ca0d91f6e&came_from=267'}

In [40]:
for j, event in enumerate(tour): 
    print (j)    
    event_info.update_one({'id':event['id']},
                         {'$set':event}, upsert=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [5]:
capstone_db = mc['bit_scraping']
event_info  = capstone_db['events']

In [8]:
event_info.delete_many({})

In [17]:
data = pd.DataFrame(list(event_info.find()))

10451

In [396]:
'%20'.join(("This Is A Test").split(' '))

'This%20Is%20A%20Test'